# <center>Basic Model for Supply Chain Transportation Lanes Using the Minimum Cost Flow Optimization Model </center>

<center>Sabi Horvat, January 2021</center>

This example model is similar to components of larger projects that I worked on from 2010-2014 with Xpress-Mosel.

What does this model do?  In summary, the model chooses transportation lanes from a number of options.  In practice, the data for these options is changing at least on a quarterly basis due to the following reasons.

* Contracts with the suppliers that manufacture these products expire and require periodic renegotiation. 
* Transportation costs from third party logistics (3PLs) fluctuate depending on fuel prices and other costs, but also due to shifting demand and supply.  One example is when a truck delivers products to an area where there is not also product to load upon a return trip.  When the 3PL can't coordinate for the truck to be highly utilized, such as in this situation, the truck is being paid to "deadhead" back empty or at other times at less than a Full TruckLoad (FTL).  These cost differences account for the increase in price for the quote you receive, compared to other destinations.

Additional python formulations of Minimum Cost Flow models can be found at: 
* [COIN-OR / PuLP (github) - American Steel Problem](https://github.com/coin-or/pulp/blob/master/examples/AmericanSteelProblem.py)
* [github-alerera - MCNF Timespace Problem](https://github.com/alerera/logistics-opt-python/blob/master/pulp/mcnf_timespace.py)
* [Gurobi-github - General Landing Page for Multiple Problems ](https://gurobi.github.io/modeling-examples/)

---
## Python Model

In [2]:
import pandas as pd
from pulp import *

## Data

In [3]:
# This data has been aggregated for period of time
# The transportation lane will be effective for this projected demand, during that time

# Data
#  Suppliers at Portland, Oregon and Portland, Maine
#  Distribution Center / Warehouse at Chicago, Illinois
#  Customer Demand in Seattle, Miami, Dallas, and San Diego

locations = ['Portland,OR','Portland,ME','Chicago,IL','Seattle,WA',
            'Miami,FL','Dallas,TX','SanDiego,CA']

supply_and_demand = {'Portland,OR':[100,0],
                     'Portland,ME':[100,0],
                     'Chicago,IL' :[0,50],
                     'Seattle,WA' :[0,40],
                     'Miami,FL'   :[0,30],
                     'Dallas,TX'  :[0,50],
                     'SanDiego,CA':[0,30]}
(supply, demand) = splitDict(supply_and_demand)

route_arcs = [   ('Portland,OR','Portland,OR'),
                 ('Portland,OR','Chicago,IL'),
                 ('Portland,OR','Seattle,WA'),
                 ('Portland,OR','Miami,FL'),
                 ('Portland,OR','Dallas,TX'),
                 ('Portland,OR','SanDiego,CA'),
                 ('Portland,ME','Portland,ME'),
                 ('Portland,ME','Chicago,IL'),
                 ('Portland,ME','Seattle,WA'),
                 ('Portland,ME','Miami,FL'),
                 ('Portland,ME','Dallas,TX'),
                 ('Portland,ME','SanDiego,CA'),
                 ('Chicago,IL','Seattle,WA'),
                 ('Chicago,IL','Miami,FL'),
                 ('Chicago,IL','Dallas,TX'),
                 ('Chicago,IL','SanDiego,CA')   ]

# cost per unit (variable per unit, fixed for utilizing lane)
route_cost = {   ('Portland,OR','Portland,OR'):[0,0],
                 ('Portland,OR','Chicago,IL'): [0.5,0],
                 ('Portland,OR','Seattle,WA'): [0.1,100],
                 ('Portland,OR','Miami,FL'):   [0.9,100],
                 ('Portland,OR','Dallas,TX'):  [0.5,100],
                 ('Portland,OR','SanDiego,CA'):[0.2,100],
                 ('Portland,ME','Portland,ME'):[0,0],
                 ('Portland,ME','Chicago,IL'): [0.4,100],
                 ('Portland,ME','Seattle,WA'): [0.9,100],
                 ('Portland,ME','Miami,FL'):   [0.3,100],
                 ('Portland,ME','Dallas,TX'):  [0.6,100],
                 ('Portland,ME','SanDiego,CA'):[0.9,100],
                 ('Chicago,IL','Seattle,WA'):  [0.5,0],
                 ('Chicago,IL','Miami,FL'):    [0.2,0],
                 ('Chicago,IL','Dallas,TX'):   [0.3,0],
                 ('Chicago,IL','SanDiego,CA'): [0.5,0]   }
(variable_costs, fixed_cost) = splitDict(route_cost)

## Model

In [4]:
# Decision Variables
flow = LpVariable.dicts("Route",route_arcs,lowBound=0,upBound=100,cat="Integer")

# Model, to minimize the objective function
model = LpProblem("Minimum_Cost_Flow_Problem_Sample",LpMinimize)

# Creates the objective function, product cost is not included in this simplified model
model += lpSum([flow[a]* variable_costs[a] for a in route_arcs]), "Transportation Cost "

# Constraint: Supply the demand
for l in locations:
    model += (supply[l]+ lpSum([flow[(i,j)] for (i,j) in route_arcs if j == l]) >=
             demand[l]+ lpSum([flow[(i,j)] for (i,j) in route_arcs if i == l])), "Flow %s"%l


## Results

In [6]:
# Solve the model
model.solve()

# The status of the solution (Optimal, Infeasible, Unbounded, Not Solved, or Undefined)
print("Status:", LpStatus[model.status])

# The objective solution
# for v in model.variables():
#     print(v.name, "=", v.varValue)

# The optimal objective function value 
print("\nTotal Cost = ", value(model.objective))

Status: Optimal

Total Cost =  66.0


In [7]:
# Objective Solution
result_value = []
for v in model.variables():
    result_value.append(v.varValue)

result_arc = []
for v in model.variables():
    result_arc.append(v.name)

result_od = pd.DataFrame(result_arc, columns=['result_string'])
result_od['result_string'] = result_od['result_string'].str.replace('Route_','')
result_od['result_string'] = result_od['result_string'].str.replace('[^\w\s]','', regex=True)
result_od[['origin','destination']] = result_od['result_string'].str.split('_',1, expand=True)
result_od['flow'] = result_value
result_od = result_od.drop(['result_string'], axis=1)[result_od['flow'] > 0]
result_od

,origin,destination,flow
4,PortlandME,ChicagoIL,50.0
5,PortlandME,DallasTX,20.0
6,PortlandME,MiamiFL,30.0
11,PortlandOR,DallasTX,30.0
14,PortlandOR,SanDiegoCA,30.0
15,PortlandOR,SeattleWA,40.0
